In [1]:
#!/usr/bin/env python
from osgeo import gdal
from osgeo import osr
osr.UseExceptions()
import numpy as np
import os, sys
from netCDF4 import Dataset

In [2]:
import os
os.environ['GDAL_DATA'] = '/usr/share/gdal'
os.environ['PROJ_LIB'] = '/usr/share/proj'

In [44]:
def rgb_geotiff(file, outfile, red, green, blue, lat, lon):
    with Dataset(file, "r", format="NETCDF4") as nc:
        red = np.array(nc.variables[red][:])
        green = np.array(nc.variables[green][:])
        blue = np.array(nc.variables[blue][:])
        lat = np.array(nc.variables[lat][:])
        lon = np.array(nc.variables[lon][:])
        image_size = red.shape

    r_pixels = np.around((( red - np.amin(red) ) / ( np.amax(red) - np.amin(red) )) * 255)
    g_pixels = np.around((( green - np.amin(green) ) / ( np.amax(green)  - np.amin(green) )) * 255)
    b_pixels = np.around((( blue - np.amin(blue) ) / ( np.amax(blue)  - np.amin(blue) )) * 255)

    # set geotransform
    nx = image_size[0]
    ny = image_size[1]
    xmin, ymin, xmax, ymax = [min(lon), min(lat), max(lon), max(lat)]
    xres = (xmax - xmin) / float(ny)
    yres = (ymax - ymin) / float(nx)
    geotransform = (xmin, xres, 0, ymax, 0, -yres)

    # create the 3-band raster file
    dst_ds = gdal.GetDriverByName('GTiff').Create(outfile, ny, nx, 3, gdal.GDT_Byte)

    dst_ds.SetGeoTransform(geotransform)    # specify coords
    srs = osr.SpatialReference()            # establish encoding
    srs.ImportFromEPSG(4326)                # WGS84 lat/long
    dst_ds.SetProjection(srs.ExportToWkt()) # export coords to file
    dst_ds.GetRasterBand(1).WriteArray(r_pixels)   # write r-band to the raster
    dst_ds.GetRasterBand(2).WriteArray(g_pixels)   # write g-band to the raster
    dst_ds.GetRasterBand(3).WriteArray(b_pixels)   # write b-band to the raster
    dst_ds.FlushCache()                     # write to disk
    dst_ds = None

In [48]:
def singleband_geotiff(file, outfile, band, lat, lon):
    with Dataset(file, "r", format="NETCDF4") as nc:
        band = np.array(nc.variables[band][:])
        lat = np.array(nc.variables[lat][:])
        lon = np.array(nc.variables[lon][:])
        image_size = band.shape

    b_pixels = np.around((( band - np.amin(band) ) / ( np.amax(band) - np.amin(band) )) * 255)
    a_pixels = band

    # set geotransform
    nx = image_size[0]
    ny = image_size[1]
    xmin, ymin, xmax, ymax = [np.amin(lon), np.amin(lat), np.amax(lon), np.amax(lat)]
    xres = (xmax - xmin) / float(ny)
    yres = (ymax - ymin) / float(nx)
    geotransform = (xmin, xres, 0, ymax, 0, -yres)

    # create the 3-band raster file
    dst_ds = gdal.GetDriverByName('GTiff').Create(outfile, ny, nx, 4, gdal.GDT_Byte)
    
    dst_ds.SetGeoTransform(geotransform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    dst_ds.SetProjection(srs.ExportToWkt())
    dst_ds.GetRasterBand(1).WriteArray(b_pixels)
    dst_ds.GetRasterBand(2).WriteArray(b_pixels)
    dst_ds.GetRasterBand(3).WriteArray(b_pixels)
    dst_ds.GetRasterBand(4).WriteArray(a_pixels)
    dst_ds.FlushCache()
    dst_ds = None

In [49]:
rgb_geotiff("test.nc", "test.tif", "RED", "GREEN", "BLUE", "lat", "lon")

In [50]:
singleband_geotiff("test.nc", "testgs.tif", "IDEPIX_SNOW_ICE", "lat", "lon")

In [53]:
import sys
print(sys.path)

['/home/jamesrunnalls/snowlines', '/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python37.zip', '/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python3.7', '/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python3.7/lib-dynload', '', '/home/jamesrunnalls/.local/lib/python3.7/site-packages', '/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python3.7/site-packages', '/home/jamesrunnalls/git/snowline', '/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python3.7/site-packages/IPython/extensions', '/home/jamesrunnalls/.ipython']


In [12]:
nc = Dataset("tests/test3.nc", "r", format="NETCDF4")

In [13]:
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.4
    TileSize: 284:420
    product_type: BandMath
    metadata_profile: beam
    metadata_version: 0.5
    start_date: 08-DEC-2020 09:03:35.511766
    stop_date: 08-DEC-2020 09:03:35.511766
    title: SENTINEL-3 OLCI Level 1 Earth Observation Full Resolution Product
    dimensions(sizes): lat(849), lon(1258)
    variables(dimensions): int8 metadata(), float32 IDEPIX_SNOW_ICE(lat,lon), float32 IDEPIX_CLOUD(lat,lon), float32 IDEPIX_INVALID(lat,lon), float32 IDEPIX_CLOUD_AMBIGUOUS(lat,lon), float32 IDEPIX_CLOUD_SURE(lat,lon), float32 IDEPIX_CLOUD_BUFFER(lat,lon), float32 IDEPIX_CLOUD_SHADOW(lat,lon), float32 IDEPIX_LAND(lat,lon), float32 RED(lat,lon), float32 GREEN(lat,lon), float32 BLUE(lat,lon), float64 lat(lat), float64 lon(lon), int32 crs()
    groups: 

In [14]:
nc.variables["RED"][:]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=nan,
  dtype=float32)

In [15]:
np.nanmin(np.array(nc.variables["RED"][:]))

/home/jamesrunnalls/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: All-NaN slice encountered
  """Entry point for launching an IPython kernel.


nan

In [16]:
np.all(red == np.nan)

False

In [20]:
np.isnan(op_list).all()

True

In [26]:
import urllib.request, json 
with urllib.request.urlopen("https://snowlines-database.s3.eu-central-1.amazonaws.com/snowline.json") as url:
    data = json.loads(url.read().decode())["data"]
    print(data)

[{'id': 1, 'datetime': 1607590235.0, 'url': 'snowline_20201210_0950.json'}, {'id': 2, 'datetime': 1607678764.0, 'url': 'snowline_20201211_1026.json'}, {'id': 3, 'datetime': 1607850781.0, 'url': 'snowline_20201213_1013.json'}, {'id': 4, 'datetime': 1607935610.0, 'url': 'snowline_20201214_0946.json'}, {'id': 5, 'datetime': 1608024139.0, 'url': 'snowline_20201215_1022.json'}, {'id': 6, 'datetime': 1608196157.0, 'url': 'snowline_20201217_1009.json'}, {'id': 7, 'datetime': 1608280986.0, 'url': 'snowline_20201218_0943.json'}, {'id': 8, 'datetime': 1608369515.0, 'url': 'snowline_20201219_1018.json'}, {'id': 9, 'datetime': 1608456704.0, 'url': 'snowline_20201220_1031.json'}, {'id': 10, 'datetime': 1608541533.0, 'url': 'snowline_20201221_1005.json'}, {'id': 11, 'datetime': 1608626363.0, 'url': 'snowline_20201222_0939.json'}, {'id': 12, 'datetime': 1608714891.0, 'url': 'snowline_20201223_1014.json'}, {'id': 13, 'datetime': 1608802081.0, 'url': 'snowline_20201224_1028.json'}, {'id': 14, 'datetime

In [24]:

with urllib.request.urlopen("https://snowlines-database.s3.eu-central-1.amazonaws.com/geotiff/20201230.json") as url:
    data = json.loads(url.read().decode())
    print(data)

HTTPError: HTTP Error 403: Forbidden